# Some graphics visualisation from our final table

### ! This code has not been cleaned. It's only use was to display interesting graphs !

In [ ]:
import sys
import os
import cv2
import json
import pandas as pd
import random
import sys
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from math import sqrt
import glob
sys.path.insert(0, "..")
from utils_analytics import *


In [ ]:
df_final = pd.read_pickle('../inputs/df_efficiency_2.pkl')

In [ ]:
df_final

In [ ]:
df_data = pd.read_pickle('../inputs/final_concreting_df.pkl')

In [ ]:
df_data = df_data[df_data['filename'].str.startswith('2020')]

In [ ]:
df_data['date_time'] =  pd.to_datetime(df_data['date_time'])

In [ ]:
df_data.head()

In [ ]:
df_data['hour'] = df_data.apply(lambda row : row['date_time'].hour, axis=1)

In [ ]:
df_data

# Evolution of number of workers

In [ ]:
a = df_data.groupby(["date_time", "classTitle"]).agg({'start_time':'count', 'period':'max'}).reset_index()
a

In [ ]:
a.period.unique()

In [ ]:
 a[(a['classTitle']=='People') & (a['period']==0.)].sort_values(by=['date_time'])

In [ ]:
for i in a.period.unique():
     a[(a['classTitle']=='People') & (a['period']==i)].sort_values(by=['date_time'])
    

In [ ]:
b=  a[(a['classTitle']=='People') & (a['period']==0.)].sort_values(by=['date_time'])
b

In [ ]:
b.start_time.unique()

In [ ]:
b.groupby(['date_time']).agg({'start_time':'mean'})


In [ ]:

plt.figure(figsize=(10,5))
plt.bar(b['date_time'], b['start_time'])
plt.title('Evolution of number of Workers over one period')
#plt.xticks(b['date_time'])
plt.xlabel('Date')
plt.ylabel('Number of workers')
plt.show()

# Evolution of worker in a day

In [ ]:
df_data.head()

In [ ]:
c = df_data.groupby(["date_time", "classTitle"]).agg({'start_time':'count', 'hour':'max'}).reset_index()
c

In [ ]:
d =  c[(c['classTitle']=='People')].sort_values(by=['hour'])
d

In [ ]:
d = d.groupby(['hour']).agg({'start_time':'mean'})
d.index

In [ ]:

plt.figure(figsize=(10,5))
plt.bar(d.index, d['start_time'])
plt.title('Evolution of number of Workers per hour')
#plt.xticks(b['date_time'])
plt.xlabel('Hour')
plt.ylabel('Number of workers')
plt.show()

# number of workers per size of concreting zone

In [ ]:
poly = df_data[df_data['classTitle']=='Concrete_pump_hose']['ext_points'][893]

In [ ]:
!pip install shapely

In [ ]:
from shapely.geometry import Polygon
Polygon(poly).length

In [ ]:
df_pump = df_data[df_data['classTitle']=='Concrete_pump_hose']

In [ ]:
df_pump.head()

In [ ]:
df_pump.apply(lambda row : Polygon(row['pump_polygon']).length if row['pump_polygon']!=0 else 0, axis=1)

In [ ]:
df_pump['size_poly'] = df_pump.apply(lambda row : Polygon(row['pump_polygon']).length if row['pump_polygon']!=0 else 0, axis=1)

In [ ]:
g = df_pump[['filename', 'size_poly']]
g

In [ ]:
e = df_data.groupby(["filename", "classTitle"]).agg({'start_time':'count', 'period':'max'}).reset_index()
f = e[e['classTitle']=='People']
f

In [ ]:
h = f.merge(g, left_on='filename', right_on='filename')
h = h[h['size_poly'] != 0]

In [ ]:
h

In [ ]:


plt.figure(figsize=(10,8))
plt.scatter(h.size_poly, h.start_time)
plt.title('Number of workers by Size of Polygon')
#plt.xticks(b['date_time'])
plt.xlabel('Size of Concrete Pump Zone')
plt.ylabel('Number of workers')
plt.show()